# Group 21 Lab 5
##### Group Members: (Andrew Quijano, afq2003), (Isaiah Genis | ig596), (Jamie Leach | jhl925), (Yevhen Pankevych | yp2525)

Your first goal as a Security Engineer who wants to ensure the security of the application is to identify security vulnerabilities in the project and create testcases to exploit them. In the real world, applications are often tested with test scripts. The following are a few questions a Security Engineer would ask when writing a testcase to test the Security of their application:

* "Does this application protect the confidentiaility of data?"
* "Does this application hamper performance?"
* "Does this application prevent actions that shoudl be allowed?"

Notice that these questions are parallel to security paradigms: Security, Efficiency and Accuracy, Availability. If you can find a hypothesis where a security paradigm is not met that would mean the application is not secure and a possible exploit exist. The testcases can be written in python or bash.

### Task:
We have listed hints for 3 vulnerabilities below which are currently present in code. Your task is to explain how it violates the security regulations (in terms of the above questions) and then provide the testcases to demonstrate how these vulnerabilities can be exploited.
* Password is Hardcoded (SampleNetworkClient.py)
* Token List has the potential to continuously grow (SampleNetworkServer.py)
* Plaintext authentication token and authentication (SampleNetworkClient.py)

### Submission:
For each vulnerability above:

1. describe how it answers the three questions mentioned above.
2. Provide test cases (in python or bash) to demonstrate how these vulnerabilities can be exploited.

### Part 1: Describe how it answers the three questions mentioned above.

* Does this application protect the confidentiality of data?"

    * No, the application cannot protect the confidentiality of data. 
        1. This is because a vulnerability was discovered in the ‘SampleNetworkClient.py’ part of the application, shown below, which contains a function that passes/stores a hardcoded password value to send it within a message to the Server side socket. Hardcoded credentials generally pose a security risk to an application as an attacker can reverse engineer the code to obtain the password to use elsewhere. In this case an attacker would be able to impersonate/seize the client’s account as their username/password has been exposed. The relevant code has been commented below and a test case has been written in Python/Bash to demonstrate how to exploit this vulnerability.

In [3]:
# SampleNetworkClient.py:
def authenticate(self, p, pw) : #pw variable here takes in raw input from user/client calling the function.
        s = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
        s.sendto(b"AUTH %s" % pw, ("127.0.0.1", p)) #pw variable here is exposed and/or is not used in authentication
        msg, addr = s.recvfrom(1024)
        return msg.strip()

* Does this application protect the confidentiality of data?"

    * No, the application cannot protect the confidentiality of data. 
        2. Additionally another vulnerability was discovered in the ‘SampleNetworkClient.py’ & ‘SampleNetworkServer.py’ part of the application, shown below, which contains plaintext authentication tokens, which are values used to identify/track valid logged in user sessions and passes these values between the Client and Server side sockets. Plaintext session tokens jeopardize active sessions within the application as an attacker can easily identify which user/session is running on the system and could potentially terminate a client’s session. This vulnerability also enables a Man-in-the-middle attack, which allows an attacker to obtain an unencrypted token and use it to impersonate and act on behalf of a legitimate user. The relevant code has been commented below and a test case has been written in Python/Bash to demonstrate how to exploit this vulnerability.

In [ ]:
#SampleNetworkClient.py:
def updateInfTemp(self, frame) :
        self.updateTime()
        if self.infToken is None : #not yet authenticated
            self.infToken = self.authenticate(self.infPort, b"!Q#E%T&U8i6y4r2w") # this token is exposed/plaintext

  def updateIncTemp(self, frame) :
        self.updateTime()
        if self.incToken is None : #not yet authenticated
            self.incToken = self.authenticate(self.incPort, b"!Q#E%T&U8i6y4r2w") # this token is exposed/plaintext

In [ ]:
#SampleNetworkServer.py:
if len(cs) == 2 : #should be either AUTH or LOGOUT
                if cs[0] == "AUTH":
                    if cs[1] == "!Q#E%T&U8i6y4r2w" : # this token is exposed/plaintext
                        self.tokens.append(''.join(random.choice(string.ascii_uppercase + string.ascii_lowercase + string.digits) for _ in range(16)))
                        self.serverSocket.sendto(self.tokens[-1].encode("utf-8"), addr)


* "Does this application hamper performance?"

    * Yes, the application currently hampers performance in several ways:
        1. The hardcoded password mentioned before currently speeds up the overall client/user experience in authentication however this poses a risk to the overall security/performance as the password is not able to be easily changed if it has been compromised. 
        2. A vulnerability was discovered in the ‘SampleNetworkServer.py’ part of the application which allows the array “self.tokens”, that is used to stores generated token values for each authenticated session, to continuously grow if authenticated sessions continually occur without ‘logouts’. This vulnerability can lead to performance issues should the list become too long and could also lead to a buffer overflow if attacked. The relevant code has been highlighted below and a test case (XXX) has been written in Python/Bash to demonstrate this vulnerability.


In [ ]:
# SampleNetworkServer.py
def processCommands(self, msg, addr) :
        cmds = msg.split(';')
        for c in cmds :
            cs = c.split(' ')
            if len(cs) == 2 : #should be either AUTH or LOGOUT
                if cs[0] == "AUTH":
                    if cs[1] == "!Q#E%T&U8i6y4r2w" :
                        self.tokens.append(''.join(random.choice(string.ascii_uppercase + string.ascii_lowercase + string.digits) for _ in range(16)))
                        #self.tokens.append is a vulnerability as there is no limit specified for the array
                        self.serverSocket.sendto(self.tokens[-1].encode("utf-8"), addr)
                        #print (self.tokens[-1])
                elif cs[0] == "LOGOUT":
                    if cs[1] in self.tokens :
                        self.tokens.remove(cs[1])


* "Does this application prevent actions that should be allowed?"

    * Yes, this system has potential vulnerabilities that can cause a refusal to perform the correct action
        1. An unencrypted token, as mentioned earlier, can be intercepted by an attacker and used to cause deauthentication of that user. Thus, a legitimate user will lose access to the system and require re-authentication.
        2. Also, the system can potentially become unavailable due to a DOS attack aimed at overflowing the list of available tokens or calling the logout procedure with any data (‘SampleNetworkServer.py’) - this is possible due to the fact that the size of the list is not fixed and can be quite large - then the time to search for an invalid token will be equal to the viewing time of all items in the list, which can cause delays in the operation of the server, up to the failure of its operation. The relevant code has been highlighted below.

In [5]:
def processCommands(self, msg, addr) :
        cmds = msg.split(';')
        for c in cmds :
            cs = c.split(' ')
            if len(cs) == 2 : #should be either AUTH or LOGOUT
                if cs[0] == "AUTH":
                    if cs[1] == "!Q#E%T&U8i6y4r2w" :
                        self.tokens.append(''.join(random.choice(string.ascii_uppercase + string.ascii_lowercase + string.digits) for _ in range(16)))
                        self.serverSocket.sendto(self.tokens[-1].encode("utf-8"), addr)
                        #print (self.tokens[-1])
                elif cs[0] == "LOGOUT":
                    if cs[1] in self.tokens :#The array is not fixed limit and could DDOS system if the list becomes too large/attacked 
                        self.tokens.remove(cs[1])
                else : #unknown command
                    self.serverSocket.sendto(b"Invalid Command\n", addr)

### Part 2

In [ ]:
#Exploit script for “Token List has the potential to continuously grow”:

import socket

while True:
    s = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
    s.sendto(b"AUTH !Q#E%T&U8i6y4r2w", ("127.0.0.1", 23456))
    msg, addr = s.recfrom(1024)
    
    s = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
    s.sendto(b"LOGOUT random-token", ("127.0.0.1", 23456))